In [0]:
from __future__ import print_function
import random as rand
import csv
import operator
import gc
import os
from datetime import datetime
from keras.callbacks import EarlyStopping
from keras.models import load_model
import keras.backend as K
from sklearn.metrics import log_loss
import numpy as np
import mlflow
import mlflow.keras

if K.backend() == 'tensorflow':
    import tensorflow as tf
from setuptools import setup, find_packages

from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras import backend as K
from devol import DEvol, GenomeHandler
import pandas as pd
from pyspark.sql.types import *
import inspect
from typing import Callable, List
import pandas as pd
from pyspark.sql import DataFrame, Row, column
from pyspark.sql.functions import lit, pandas_udf, PandasUDFType, array
from pyspark.sql.types import FloatType

In [0]:
import pickle 
pickle_file = open('/dbfs/FileStore/models/simple/pickle','rb')
retrieved_devol = pickle.load(pickle_file)


In [0]:
#use rdd to parallelize training
def transform_row(row):
  model = None
  loss = None
  accurcacy = None
  accuracy = 0
  gene = row.v
  gene = gene.split(sep = ',')
  result = []
  for item in gene:
    result.append(int(item))
    
  if(genome_handler.is_compatible_genome(result)):
    model, loss, accuracy = retrieved_devol._evaluate(result,1)
  return model, loss, accuracy
lam = lambda row: transform_row(row)


In [0]:
if(len(tf.config.experimental.list_physical_devices('GPU')) > 0):
  print("GPU accelerated")
  print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
else:
  print("CPU mode")

GPU accelerated
Num GPUs Available: 2

In [0]:
"""
Run a genetic algorithm to find an appropriate architecture for some image
classification task with Keras+TF.
To use, define a `GenomeHandler` defined in genomehandler.py. Then pass it, with
training data, to a DEvol instance to run the genetic algorithm. See the readme
for more detailed instructions.
"""

from __future__ import print_function
import random as rand
import csv
import operator
import gc
import os
from datetime import datetime
from keras.callbacks import EarlyStopping
from keras.models import load_model
import keras.backend as K
from sklearn.metrics import log_loss
import numpy as np

if K.backend() == 'tensorflow':
    import tensorflow as tf

__all__ = ['DEvol']

METRIC_OPS = [operator.__lt__, operator.__gt__]
METRIC_OBJECTIVES = [min, max]

dbutils.fs.mkdirs( '/dbfs/FileStore/models/gpu/')


class DEvol:
    """
    Object which carries out genetic search and returns top performing model
    upon completion.
    """
#!!! data_path with checkpointing
    def __init__(self, genome_handler, data_path="/dbfs/FileStore/runs/"):
        """
        Initialize a DEvol object which carries out the training and evaluation
        of a genetic search.
        Args:
            genome_handler (GenomeHandler): the genome handler object defining
                    the restrictions for the architecture search space
            data_path (str): the file which the genome encodings and metric data
                    will be stored in
        """
        self.data_path = data_path + datetime.now().ctime().replace(":","-").replace(" ","_")
        data_path = self.data_path
        dbutils.fs.mkdirs(data_path )
        
        self.genome_handler = genome_handler
        self.datafile = data_path or (data_path + 'record.csv')
        self._bssf = -1



        if os.path.isfile(data_path) and os.stat(data_path).st_size > 1:
            raise ValueError(('Non-empty file %s already exists. Please change'
                              'file path to prevent overwritten genome data.'
                              % data_path))

        print("Genome encoding and metric data stored at", self.datafile, "\n")
        with open(self.datafile, 'a') as csvfile:
            writer = csv.writer(csvfile, delimiter=',', quotechar='"',
                                quoting=csv.QUOTE_MINIMAL)
            metric_cols = ["Val Loss", "Val Accuracy"]
            genome = genome_handler.genome_representation() + metric_cols
            writer.writerow(genome)

    def set_objective(self, metric):
        """
        Set the metric for optimization. Can also be done by passing to
        `run`.
        Args:
            metric (str): either 'acc' to maximize classification accuracy, or
                    else 'loss' to minimize the loss function
        """
        if metric == 'acc':
            metric = 'accuracy'
        if metric not in ['loss', 'accuracy']:
            raise ValueError(('Invalid metric name {} provided - should be'
                              '"accuracy" or "loss"').format(metric))
        self._metric = metric
        self._objective = "max" if self._metric == "accuracy" else "min"
        self._metric_index = 1 if self._metric == 'loss' else -1
        self._metric_op = METRIC_OPS[self._objective == 'max']
        self._metric_objective = METRIC_OBJECTIVES[self._objective == 'max']
       
        
    def run(self, dataset, num_generations, pop_size, epochs, fitness=None,
            metric='accuracy'):
        """
        Run genetic search on dataset given number of generations and
        population size
        Args:
            dataset : tuple or list of numpy arrays in form ((train_data,
                    train_labels), (validation_data, validation_labels))
            num_generations (int): number of generations to search
            pop_size (int): initial population size
            epochs (int): epochs for each model eval, passed to keras model.fit
            fitness (None, optional): scoring function to be applied to
                    population scores, will be called on a numpy array which is
                    a min/max scaled version of evaluated model metrics, so It
                    should accept a real number including 0. If left as default
                    just the min/max scaled values will be used.
            metric (str, optional): must be "accuracy" or "loss" , defines what
                    to optimize during search
        Returns:
            keras model: best model found with weights
        """
        self.set_objective(metric)

        # If no validation data is given set it to None
        if len(dataset) == 2:
            (self.x_train, self.y_train), (self.x_test, self.y_test) = dataset
            self.x_val = None
            self.y_val = None
        else:
            (self.x_train, self.y_train), (self.x_test, self.y_test), (self.x_val, self.y_val) = dataset

        # generate and evaluate initial population
        members = self._generate_random_population(pop_size)
        pop = self._evaluate_population(members,
                                        epochs,
                                        fitness,
                                        0,
                                        num_generations)

        # evolve
        for gen in range(1, num_generations):
            members = self._reproduce(pop, gen)
            #!!! map to pandas df, apply udf parallel training, save scores
            pop = self._evaluate_population(members,
                                            epochs,
                                            fitness,
                                            gen,
                                            num_generations)
        
        #!!!add checkpointing to dbfs

        return 1
        #return load_model('best-model.h5')

    def _reproduce(self, pop, gen):
        members = []

        # 95% from crossover
        for _ in range(int(len(pop) * 0.95)):
            members.append(self._crossover(pop.select(), pop.select()))

        # best models survive automatically
        members += pop.get_best(len(pop) - int(len(pop) * 0.95))

        # randomly mutate
        for imem, mem in enumerate(members):
            members[imem] = self._mutate(mem, gen)
        return members

    def _evaluate(self, genome, epochs):
        model = self.genome_handler.decode(genome)
        loss, accuracy = None, None
        fit_params = {
            'x': self.x_train,
            'y': self.y_train,
            'validation_split': 0.1,
            'epochs': epochs,
            'verbose': 1,
            'callbacks': [
                EarlyStopping(monitor='val_loss', patience=1, verbose=1)
            ]
        }

        if self.x_val is not None:
            fit_params['validation_data'] = (self.x_val, self.y_val)
        try:
            model.fit(**fit_params)
            loss, accuracy = model.evaluate(self.x_test, self.y_test, verbose=0)
        except Exception as e:
            loss, accuracy = self._handle_broken_model(model, e)

        self._record_stats(model, genome, loss, accuracy)

        return model, loss, accuracy

    def _record_stats(self, model, genome, loss, accuracy):
        with open(self.datafile, 'a') as csvfile:
            writer = csv.writer(csvfile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
            row = list(genome) + [loss, accuracy]
            writer.writerow(row)

        met = loss if self._metric == 'loss' else accuracy
        if (self._bssf is -1 or
                self._metric_op(met, self._bssf) and
                accuracy is not 0):
            
            self._bssf = met
            #azure enforces mlflow saving
            mlflow.keras.save_model(model, self.data_path + '/best-model-mlflow')

    def _handle_broken_model(self, model, error):
        del model

        n = self.genome_handler.n_classes
        loss = log_loss(np.concatenate(([1], np.zeros(n - 1))), np.ones(n) / n)
        accuracy = 1 / n
        gc.collect()

        if K.backend() == 'tensorflow':
            K.clear_session()
            tf.reset_default_graph()

        print('An error occurred and the model could not train:')
        print(error)
        print(('Model assigned poor score. Please ensure that your model'
               'constraints live within your computational resources.'))
        return loss, accuracy

    def  p_eval(self, df1, epochs):
      #use rdd to parallelize training
      
      lam = lambda row: transform_row(row)

      loss_acc_rdd = df1.rdd.map(lam).collect()
      return loss_acc_rdd
    
    def _evaluate_population(self, members, epochs, fitness, igen, ngen):
        df1 = pd.DataFrame()
        list= []
        for member in members:
            gene = (str(member))[1:-1] 
            sep = ''
            foo=(sep.join(gene))
            list.append(foo)
            print(foo)

        df1 = pd.DataFrame(list)
        df1 = spark.createDataFrame(df1, [ "v"])
        
        result = self.p_eval(df1, epochs)
        
        
        fit = []
        #data_path = "/dbfs/FileStore/test/"
        #result2
        for imem, mem in enumerate(members):
          model = result[imem][0]
          loss = result[imem][1]
          accuracy = result[imem][2]
          name = datetime.now().ctime() + "_" + str(accuracy) + "_" +str(loss)
          
          mlflow.keras.save_model(model,(self.data_path+ name))
          #model.save_model(data_path + name)
          print("saved :",name,"\n")
          res = model, loss, accuracy
          v = res[self._metric_index]
          print(res[self._metric_index])
          del res
          fit.append(v)
        
        fit = np.array(fit)
        self._print_result(fit, igen)
        return _Population(members, fit, fitness, obj=self._objective)

    def _print_evaluation(self, imod, nmod, igen, ngen):
        fstr = '\nmodel {0}/{1} - generation {2}/{3}:\n'
        print(fstr.format(imod + 1, nmod, igen + 1, ngen))

    def _generate_random_population(self, size):
        return [self.genome_handler.generate() for _ in range(size)]

    def _print_result(self, fitness, generation):
        result_str = ('Generation {3}:\t\tbest {4}: {0:0.4f}\t\taverage:'
                      '{1:0.4f}\t\tstd: {2:0.4f}')
        print(result_str.format(self._metric_objective(fitness),
                                np.mean(fitness),
                                np.std(fitness),
                                generation + 1, self._metric))

    def _crossover(self, genome1, genome2):
        cross_ind = rand.randint(0, len(genome1))
        child = genome1[:cross_ind] + genome2[cross_ind:]
        return child

    def _uniform_crossover(self,genome1,genome2):
        prob_xover = 0.5
        for i in range(len(genome1)):
          if rand.random() < prob_xover:
            genome1[i], genome2[i] = genome2[i], genome1[i]
        return genome1, genome2
      
    def _mutate(self, genome, generation):
        # increase mutations as program continues
        num_mutations = max(3, generation // 4)
        return self.genome_handler.mutate(genome, num_mutations)


class _Population(object):

    def __len__(self):
        return len(self.members)

    def __init__(self, members, fitnesses, score, obj='max'):
        self.members = members
        scores = fitnesses - fitnesses.min()
        if scores.max() > 0:
            scores /= scores.max()
        if obj == 'min':
            scores = 1 - scores
        if score:
            self.scores = score(scores)
        else:
            self.scores = scores
        self.s_fit = sum(self.scores)

    def get_best(self, n):
        combined = [(self.members[i], self.scores[i])
                    for i in range(len(self.members))]
        sorted(combined, key=(lambda x: x[1]), reverse=True)
        return [x[0] for x in combined[:n]]

    
      
    #fitness proportional selection
    def select(self):
        dart = rand.uniform(0, self.s_fit)
        sum_fits = 0
        for i in range(len(self.members)):
            sum_fits += self.scores[i]
            if sum_fits >= dart:
                return self.members[i]

In [0]:
import numpy as np
import random as rand
import math
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization


class GenomeHandler:
    """
    Defines the configuration and handles the conversion and mutation of
    individual genomes. Should be created and passed to a `DEvol` instance.
    ---
    Genomes are represented as fixed-with lists of integers corresponding
    to sequential layers and properties. A model with 2 convolutional layers
    and 1 dense layer would look like:
    [<conv layer><conv layer><dense layer><optimizer>]
    The makeup of the convolutional layers and dense layers is defined in the
    GenomeHandler below under self.convolutional_layer_shape and
    self.dense_layer_shape. <optimizer> consists of just one property.
    """

    def __init__(self, max_conv_layers, max_dense_layers, max_filters,
                 max_dense_nodes, input_shape, n_classes,
                 batch_normalization=True, dropout=True, max_pooling=True,
                 optimizers=None, activations=None):
        """
        Creates a GenomeHandler according
        Args:
            max_conv_layers: The maximum number of convolutional layers
            max_dense_layers: The maximum number of dense (fully connected)
                    layers, including output layer
            max_filters: The maximum number of conv filters (feature maps) in a
                    convolutional layer
            max_dense_nodes: The maximum number of nodes in a dense layer
            input_shape: The shape of the input
            n_classes: The number of classes
            batch_normalization (bool): whether the GP should include batch norm
            dropout (bool): whether the GP should include dropout
            max_pooling (bool): whether the GP should include max pooling layers
            optimizers (list): list of optimizers to be tried by the GP. By
                    default, the network uses Keras's built-in adam, rmsprop,
                    adagrad, and adadelta
            activations (list): list of activation functions to be tried by the
                    GP. By default, relu and sigmoid.
        """
        if max_dense_layers < 1:
            raise ValueError(
                "At least one dense layer is required for softmax layer"
            )
        if max_filters > 0:
            filter_range_max = int(math.log(max_filters, 2)) + 1
        else:
            filter_range_max = 0
        self.optimizer = optimizers or [
            'adam',
            'rmsprop',
            'adagrad',
            'adadelta'
        ]
        self.activation = activations or [
            'relu',
            'sigmoid',
        ]
        self.convolutional_layer_shape = [
            "active",
            "num filters",
            "batch normalization",
            "activation",
            "dropout",
            "max pooling",
        ]
        self.dense_layer_shape = [
            "active",
            "num nodes",
            "batch normalization",
            "activation",
            "dropout",
        ]
        self.layer_params = {
            "active": [0, 1],
            "num filters": [2**i for i in range(3, filter_range_max)],
            "num nodes": [2**i for i in range(4, int(math.log(max_dense_nodes, 2)) + 1)],
            "batch normalization": [0, (1 if batch_normalization else 0)],
            "activation": list(range(len(self.activation))),
            "dropout": [(i if dropout else 0) for i in range(11)],
            "max pooling": list(range(3)) if max_pooling else 0,
        }

        self.convolution_layers = max_conv_layers
        self.convolution_layer_size = len(self.convolutional_layer_shape)
        self.dense_layers = max_dense_layers - 1  # this doesn't include the softmax layer, so -1
        self.dense_layer_size = len(self.dense_layer_shape)
        self.input_shape = input_shape
        self.n_classes = n_classes

    def convParam(self, i):
        key = self.convolutional_layer_shape[i]
        return self.layer_params[key]

    def denseParam(self, i):
        key = self.dense_layer_shape[i]
        return self.layer_params[key]

    def mutate(self, genome, num_mutations):
        num_mutations = np.random.choice(num_mutations)
        for i in range(num_mutations):
            index = np.random.choice(list(range(1, len(genome))))
            if index < self.convolution_layer_size * self.convolution_layers:
                if genome[index - index % self.convolution_layer_size]:
                    range_index = index % self.convolution_layer_size
                    choice_range = self.convParam(range_index)
                    genome[index] = np.random.choice(choice_range)
                elif rand.uniform(0, 1) <= 0.01:  # randomly flip deactivated layers
                    genome[index - index % self.convolution_layer_size] = 1
            elif index != len(genome) - 1:
                offset = self.convolution_layer_size * self.convolution_layers
                new_index = (index - offset)
                present_index = new_index - new_index % self.dense_layer_size
                if genome[present_index + offset]:
                    range_index = new_index % self.dense_layer_size
                    choice_range = self.denseParam(range_index)
                    genome[index] = np.random.choice(choice_range)
                elif rand.uniform(0, 1) <= 0.01:
                    genome[present_index + offset] = 1
            else:
                genome[index] = np.random.choice(list(range(len(self.optimizer))))
        return genome

    def decode(self, genome):
        if not self.is_compatible_genome(genome):
            raise ValueError("Invalid genome for specified configs")
        model = Sequential()
        dim = 0
        offset = 0
        if self.convolution_layers > 0:
            dim = min(self.input_shape[:-1])  # keep track of smallest dimension
        input_layer = True
        for i in range(self.convolution_layers):
            if genome[offset]:
                convolution = None
                if input_layer:
                    convolution = Convolution2D(
                        genome[offset + 1], (3, 3),
                        padding='same',
                        input_shape=self.input_shape
                    )
                    input_layer = False
                else:
                    convolution = Convolution2D(
                        genome[offset + 1], (3, 3),
                        padding='same'
                    )
                model.add(convolution)
                if genome[offset + 2]:
                    model.add(BatchNormalization())
                model.add(Activation(self.activation[genome[offset + 3]]))
                model.add(Dropout(float(genome[offset + 4] / 20.0)))
                max_pooling_type = genome[offset + 5]
                # must be large enough for a convolution
                if max_pooling_type == 1 and dim >= 5:
                    model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
                    dim = int(math.ceil(dim / 2))
            offset += self.convolution_layer_size

        if not input_layer:
            model.add(Flatten())

        for i in range(self.dense_layers):
            if genome[offset]:
                dense = None
                if input_layer:
                    dense = Dense(genome[offset + 1], input_shape=self.input_shape)
                    input_layer = False
                else:
                    dense = Dense(genome[offset + 1])
                model.add(dense)
                if genome[offset + 2]:
                    model.add(BatchNormalization())
                model.add(Activation(self.activation[genome[offset + 3]]))
                model.add(Dropout(float(genome[offset + 4] / 20.0)))
            offset += self.dense_layer_size

        model.add(Dense(self.n_classes, activation='softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=self.optimizer[genome[offset]],
                      metrics=["accuracy"])
        return model

    def genome_representation(self):
        encoding = []
        for i in range(self.convolution_layers):
            for key in self.convolutional_layer_shape:
                encoding.append("Conv" + str(i) + " " + key)
        for i in range(self.dense_layers):
            for key in self.dense_layer_shape:
                encoding.append("Dense" + str(i) + " " + key)
        encoding.append("Optimizer")
        return encoding

    def generate(self):
        genome = []
        for i in range(self.convolution_layers):
            for key in self.convolutional_layer_shape:
                param = self.layer_params[key]
                genome.append(np.random.choice(param))
        for i in range(self.dense_layers):
            for key in self.dense_layer_shape:
                param = self.layer_params[key]
                genome.append(np.random.choice(param))
        genome.append(np.random.choice(list(range(len(self.optimizer)))))
        genome[0] = 1
        return genome

    def is_compatible_genome(self, genome):
        expected_len = self.convolution_layers * self.convolution_layer_size \
            + self.dense_layers * self.dense_layer_size + 1
        if len(genome) != expected_len:
            return False
        ind = 0
        for i in range(self.convolution_layers):
            for j in range(self.convolution_layer_size):
                if genome[ind + j] not in self.convParam(j):
                    return False
            ind += self.convolution_layer_size
        for i in range(self.dense_layers):
            for j in range(self.dense_layer_size):
                if genome[ind + j] not in self.denseParam(j):
                    return False
            ind += self.dense_layer_size
        if genome[ind] not in range(len(self.optimizer)):
            return False
        return True

    def best_genome(self, csv_path, metric="accuracy", include_metrics=True):
        best = max if metric is "accuracy" else min
        col = -1 if metric is "accuracy" else -2
        data = np.genfromtxt(csv_path, delimiter=",")
        row = list(data[:, col]).index(best(data[:, col]))
        genome = list(map(int, data[row, :-2]))
        if include_metrics:
            genome += list(data[row, -2:])
        return genome

    def decode_best(self, csv_path, metric="accuracy"):
        return self.decode(self.best_genome(csv_path, metric, False))

# DEvol On MNIST Results
### Genome Record

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
K.set_image_data_format("channels_last")

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
dataset = ((x_train, y_train), (x_test, y_test))

In [0]:
genome_handler = GenomeHandler(max_conv_layers=6, 
                               max_dense_layers=2, # includes final dense layer
                               max_filters=256,
                               max_dense_nodes=1024,
                               input_shape=x_train.shape[1:],
                               n_classes=10)

In [0]:
devol = DEvol(genome_handler)
devol.run(dataset=dataset,
                  num_generations=10,
                  pop_size=4,
                  epochs=1)

Genome encoding and metric data stored at /dbfs/FileStore/runs/Sat_Nov_28_00-58-29_2020 

1, 16, 1, 0, 10, 1, 0, 128, 1, 0, 6, 2, 0, 256, 0, 1, 2, 2, 1, 32, 1, 1, 6, 1, 0, 64, 1, 0, 3, 0, 0, 64, 1, 1, 3, 1, 0, 32, 1, 0, 5, 0
1, 32, 1, 1, 1, 2, 1, 256, 1, 1, 4, 2, 0, 256, 1, 0, 6, 0, 0, 128, 1, 1, 4, 1, 0, 256, 0, 1, 4, 0, 0, 32, 1, 1, 6, 2, 1, 1024, 1, 1, 5, 3
1, 64, 0, 1, 0, 1, 0, 32, 0, 0, 4, 1, 1, 32, 1, 1, 4, 1, 0, 64, 0, 1, 0, 0, 0, 32, 0, 0, 4, 2, 1, 64, 1, 0, 7, 1, 0, 32, 0, 1, 8, 0
1, 128, 0, 0, 7, 0, 1, 64, 0, 0, 2, 0, 0, 8, 1, 0, 1, 0, 0, 32, 1, 1, 0, 0, 0, 16, 0, 1, 7, 0, 0, 128, 1, 0, 8, 2, 1, 256, 0, 0, 3, 3

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4275497423460332> in <module> 
 3 num_generations = 10 , 
 4 pop_size = 4 , 
 ----> 5 epochs=1)
 
 <command-4275497423459875> in run (self, dataset, num_generations, pop_size, epochs, fitness, metric) 
 127 fitness , 
 128 0 , 
 --> 129 num_generations)
 130 
 131 # evolve 

 <command-4275497423459875> in _evaluate_population (self, members, epochs, fitness, igen, ngen) 
 240 df1 = spark . createDataFrame ( df1 , [ "v" ] ) 
 241 
 --> 242 result = self . p_eval ( df1 , epochs ) 
 243 
 244 

 <command-4275497423459875> in p_eval (self, df1, epochs) 
 224 lam = lambda row : transform_row ( row ) 
 225 
 --> 226 loss_acc_rdd = df1 . rdd . map ( lam ) . collect ( ) 
 227 return loss_acc_rdd
 228 

 /databricks/spark/python/pyspark/rdd.py in collect (self) 
 829 # Default path used in OSS Spark / for non-credential passthrough clusters: 
 830 with SCCallSiteSync ( self . context ) as css : 
 --> 831 sock_info = self . ctx . _jvm . PythonRDD . collectAndServe ( self . _jrdd . rdd ( ) ) 
 832 return list ( _load_from_socket ( sock_info , self . _jrdd_deserializer ) ) 
 833 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 4 times, most recent failure: Lost task 0.3 in stage 34.0 (TID 62, 10.139.64.12, executor 13): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 480, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 472, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 513, in dump_stream
 write_int(len(bytes), stream)
 File "/databricks/spark/python/pyspark/serializers.py", line 837, in write_int
 stream.write(struct.pack("!i", value))
struct.error: 'i' format requires -2147483648 <= number <= 2147483647

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:540)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:676)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:659)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:494)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:59)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:104)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:48)
	at scala.collection.TraversableOnce$class.to(TraversableOnce.scala:310)
	at org.apache.spark.InterruptibleIterator.to(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce$class.toBuffer(TraversableOnce.scala

Genome encoding and metric data stored at /dbfs/FileStore/runs/Sat_Nov_28_01-44-49_2020 

1, 8, 0, 1, 8, 1, 1, 8, 1, 0, 4, 2, 0, 8, 1, 1, 7, 2, 0, 16, 0, 1, 6, 0, 0, 32, 0, 1, 1, 1, 1, 64, 1, 0, 3, 0, 1, 64, 1, 0, 9, 2
1, 128, 0, 1, 3, 2, 1, 32, 1, 1, 8, 1, 0, 8, 1, 0, 8, 2, 0, 256, 0, 1, 1, 1, 0, 64, 0, 0, 8, 2, 1, 32, 1, 0, 2, 0, 0, 16, 0, 0, 7, 2
1, 256, 0, 1, 2, 1, 0, 32, 1, 1, 3, 1, 1, 256, 0, 1, 10, 1, 1, 256, 0, 1, 7, 1, 0, 64, 1, 1, 8, 2, 0, 64, 1, 0, 10, 1, 1, 1024, 1, 1, 7, 2
1, 64, 1, 0, 0, 1, 0, 8, 0, 0, 3, 1, 1, 16, 0, 0, 3, 0, 0, 256, 0, 1, 6, 1, 0, 64, 0, 0, 2, 2, 0, 16, 1, 0, 6, 1, 0, 128, 1, 1, 9, 3